# 禁忌搜索解决旅行商问题
这是禁忌搜索初体验，很大程度上参照了别人的教程。

## 1、旅行商问题（TSP）
### 1.1、概述
某一旅行商从A地出发，他需要走遍A、B、C、D四地做生意并最终回到A地，求问如何探索出一条最合适的路径，使得他可以走最少的路完成任务？

### 1.2、问题建模
我们使用一个**带权的有向图**来表示整个地图。鉴于没有人会从一个地方绕个圈子回到这个地方，所以所有结点都不设自回路；如果从A地到B地之间有多条路的话，任何情况下都应该选择长度短的那一条，因此其他的路也没有必要考虑。因此，我们就使用一个简单的邻接矩阵来表示地图，结点为A、B、C、D四个城市，用<A,B>这样的结点的序偶来表示有向边（也就是城市之间的单行道）。

为了后面方便，我们约定一个巨大的数来标记不可达的路。

我们在Python里面可以使用`numpy`来处理矩阵，定义一个`class Map`来表示整个地图。为了让地图更加形象，我们编写一个函数来生成可视化脚本。可视化采用Graphviz来渲染图像。

In [ ]:
'''编者按：这些代码看看就好，别去真的运行它，否则我也不知道jupyter能给我发出多少警告。要运行请移步src文件夹。'''
import numpy as np

"""先定义了两个异常，虽然我也不知道以后会不会有用。"""
class Error(Exception):
    """Cities and pathes matrix are not conpatible, or matrix are not square matrix."""
    def __init__(self, args: object) -> None:
        self.args=args


class MapError(Error):
    def __init__(self, message:str) -> None:
        self.message = message


class Map:
    """Acturally, Map is a directed graph, which uses tuple as verteces, ndarray as edges."""
    
    # cities是结点集合，pathes是邻接矩阵。
    def __init__(self, cities:tuple, pathes:np.ndarray) -> None:

        # 两个if都是为了判定结点集合和邻接矩阵是不是匹配的，以及这个矩阵能不能拿来作为邻接矩阵。
        if pathes.shape[0] != pathes.shape[1]:
            raise MapError("Pathes matrix is not a square matrix.")
        elif len(cities) != pathes.shape[0]:
            raise MapError("Cities and Pathes are not compatible.")

        self.cities=cities
        self.pathes=pathes

    # 结点集合的接口
    def vertex(self)->tuple:
        return self.cities

    # 邻接矩阵的接口
    def edges(self)->np.ndarray:
        return self.pathes

    # 这是为了方便打印，虽然我也不知道做了可视化之后为什么还要打印。。。
    def __str__(self) -> str:
        return("Cities:\n"+str(self.cities)+"\nPathes:\n"+str(self.pathes))

In [ ]:
'''可视化函数，会把脚本内容写在rsrc文件夹内的Map.dot里面，再使用Graphviz渲染出来就好了'''
# 这两个字符串是dot语言里结点声明语句的模板
vertex="\t{name}[shape=circle, label={label}, size=0.1, fillcolor=black, style=filled, fontcolor=white]\n"
arrow="\t{v1}->{v2}[label=\"{w}\"]\n"

# 可视化一个Map
def visualize_Map(map:Map):
    # Open script file
    with open("./rsrc/Map.dot","w") as script:
        script.write("digraph {\n")
        script.write("\tlabel=\"Map\"\n")

        # Generate verteces
        verteces=list()
        for v in map.vertex():
            verteces.append(str(v))
            script.write(vertex.format(name=str(v), label=str(v)))

        # Connect
        for i in range(map.edges().shape[0]):
            for j in range(map.edges().shape[1]):
                weight = map.edges()[i,j]
                if weight!=0:
                    script.write(arrow.format(v1=verteces[i], v2=verteces[j], w=weight))

        script.write("}")

    print("Visualized successfully.")

## 2、禁忌搜索（TS）
### 2.1、概述
禁忌搜索是由爬山算法改良来的，首先需要满足爬山法的需求：
- 初始解；
- 可以由一个解得到附近的其他解的函数；
- 可以评价解的优劣的函数；

但TS对这个方法做出了改进，我们从里面引入了：
- 禁忌表（TabuList），把局部最优解记录进里面，之后有意识避开。
- 禁忌长度，禁忌表的大小限制。
- 停止规则，满足要求或者是超时了就应该停下来了。

### 2.2、TS的实现
建立了一个`class Tabu_Search`来打包概述内说的数据和方法。这个类的对象需要在构造的时候接收一个`Map`对象以及一个初始解，之后由一个迭代求解的函数来执行整个过程。

#### 2.2.1、初始解
由之前对于图的约定可知，此图是简单图，用结点来表示边就足够了。所以每一条可能的路都无非就是规定了先去哪个地方后去哪个地方，也就是说，这个问题的解可以表达为一个结点序列。因为我们之前都使用字母来命名这些城市，所以可以选择字符串来作为解的形式，诸如于`"ABCD"`，我们就用这个作为初始解。

#### 2.2.2、迭代求解函数`Find_the_Way()`
这个函数基本上没遵循啥命名规范，但鉴于这个类不会有太多函数，所以暂且就这样罢。。。
这个函数会总理算法，体现禁忌搜索。其作用是在一系列条件下给出禁忌搜索能找到的最优解。要求把Map对象输入进去，就能得到禁忌搜索的解。之后完善算法之后可以考虑改良它。

#### 2.2.3、附近解函数`neighbours()`
因为附近的解不止一个，所以这个函数需要返回解的元组。
此外，这个函数应当输出的时候会避开禁忌表内的元素以及已经判断过的解。

#### 2.2.4、评价函数`Length()`
输入一个解，对应给出一个评价，让我们可以比较解之间的优劣。我们在这里使用总路程作为评价标准，相应的这个函数也就输出路程。在更复杂的问题里面，我们会需要更复杂的评价方法，比如使用标准差、期望等等作为评价标准。

#### 2.2.5、禁忌表
很简单，一个解的列表，其被限制在**禁忌长度**的范围内。

#### 2.2.6、停止规则
停止规则准备是限制步数，这个步数会放的比较大，到时候会使用函数来观察优劣-步数之间的走势。如果步数增加的时候优劣变化已经不明显，我们就认为这个步数长度可以作为停止规则了。